# Convert HRRR GRIB2 (on AWS) to Zarr

In [ ]:
import fsspec
import xarray as xr
import hvplot.xarray
import metpy
import numpy as np

In [ ]:
fs = fsspec.filesystem('s3', anon=True)

In [ ]:
url = 'noaa-hrrr-pds'   # forecast data

In [ ]:
url = 'hrrrzarr'   # hrrr in Zarr!

In [ ]:
url = 'noaa-hrrr-bdp-pds'  # archive

In [ ]:
fs.ls('s3://hrrrzarr/sfc/')[:5]

In [ ]:
fs.ls('s3://hrrrzarr/sfc/20191105')[:5]

In [ ]:
fs.ls('s3://hrrrzarr/sfc/20191105/20191105_00z_fcst.zarr')

In [ ]:
file_location = 's3://hrrrzarr/sfc/20201216/20201216_01z_fcst.zarr'
ds = xr.open_zarr(fsspec.get_mapper(file_location, anon=True), consolidated=True)

In [ ]:
file_location = 's3://hrrrzarr/sfc/20201216/20201216_01z_fcst.zarr'
ds = xr.open_zarr(fsspec.get_mapper(file_location, anon=False, requester_pays=True), consolidated=True)

In [ ]:
fs.ls('s3://hrrrzarr/sfc/20201216/20201216_01z_fcst.zarr')[:5]

In [ ]:
a = fs.glob(url+'/hrrr.20170102/conus/hrrr.t00z.wrfnatf*.grib2')
a

In [ ]:
a = fs.glob(url+'/hrrr.20181220/conus/hrrr.t*.wrfnatf00.grib2')
a

In [ ]:
file = fsspec.open(a[0])

In [ ]:
tmp_file = fsspec.open_local('simplecache::s3://noaa-hrrr-bdp-pds/hrrr.20170125/conus/hrrr.t07z.wrfsubhf1530.grib2', 
                              s3=dict(anon=True), simplecache={'cache_storage': '/tmp'})

In [ ]:
ds = xr.open_dataset(tmp_file, engine='cfgrib', 
                       backend_kwargs=dict(filter_by_keys={'typeOfLevel': 'heightAboveGround', 'level': 2}))

In [ ]:
b = fsspec.open_local('simplecache::s3://noaa-hrrr-bdp-pds/hrrr.20170125/conus/hrrr.t08z.wrfsubhf1530.grib2', 
                              s3=dict(anon=True), simplecache={'cache_storage': './files'})

In [ ]:
b

In [ ]:
dsa = xr.open_mfdataset([a,b], engine='cfgrib', 
                       backend_kwargs=dict(filter_by_keys={'typeOfLevel': 'heightAboveGround', 
                                                           'level': 2}))

In [ ]:
ds = xr.open_dataset(a, engine='cfgrib')

In [ ]:
print(ds.data_vars)

In [ ]:
ds = ds.metpy.parse_cf()

In [ ]:
crs = ds.t2m.metpy.cartopy_crs

In [ ]:
ds_uv = xr.open_dataset(a, engine='cfgrib', 
                        backend_kwargs=dict(filter_by_keys={'typeOfLevel': 'heightAboveGround', 
                                                            'level': 10,
                                                            'stepType': 'instant'}))

In [ ]:
ds_uv

In [ ]:
ds['u10'] = ds_uv.u10
ds['v10'] = ds_uv.v10

In [ ]:
ds.data_vars

In [ ]:
dsn = xr.open_dataset(a, engine='cfgrib', 
                        backend_kwargs=dict(filter_by_keys={'typeOfLevel': 'surface', 
                                                            'stepType': 'instant'}))

In [ ]:
dsn.data_vars

In [ ]:
ds.u10.hvplot.quadmesh(x='longitude', y='latitude', rasterize=True,
                      crs=crs, tiles='OSM', project=True, alpha=0.5, cmap='viridis')

```
# download grib2 files
$ wget https://noaa-hrrr-bdp-pds.s3.amazonaws.com/hrrr.20170125/conus/hrrr.t07z.wrfsubhf1530.grib2
$ wget https://noaa-hrrr-bdp-pds.s3.amazonaws.com/hrrr.20170125/conus/hrrr.t08z.wrfsubhf1530.grib2
$ wget https://noaa-hrrr-bdp-pds.s3.amazonaws.com/hrrr.20170125/conus/hrrr.t09z.wrfsubhf1530.grib2

# convert grib2 to standard netcdf using wgrib2
$ wgrib2 hrrr.t07z.wrfsubhf1530.grib2  -netcdf hrrr.t07z.wrfsubhf1530.nc
$ wgrib2 hrrr.t08z.wrfsubhf1530.grib2  -netcdf hrrr.t08z.wrfsubhf1530.nc
$ wgrib2 hrrr.t09z.wrfsubhf1530.grib2  -netcdf hrrr.t09z.wrfsubhf1530.nc
```

In [ ]:
ds = xr.open_mfdataset('hrrr*.nc', chunks={'time':72, 'x':600, 'y':600})

In [ ]:
ds.USWRF_surface.attrs

In [ ]:
ds.data_vars

In [ ]:
ds